In [1]:
#!conda install transformers

import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
#import tensorflow_hub as hub
from transformers import BertTokenizer
#!pip install tensorflow_text
#import tensorflow_text as text
import numpy as np
df = pd.read_csv('unprocessed_lyrics.csv')
df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(1000, random_state=42)) #


### Initializing empty arrays for storing tokenized text

In [2]:

possible_labels = df.Genre.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

df['label'] = df.Genre.replace(label_dict)

In [3]:
# set array dimensions
seq_len = 512 # Our input sequences can't be too long so we set it to max 512 tokens
num_samples = len(df) 

# initialize empty zero arrays
Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))

# check shape
print(Xids.shape)

# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i, lyric in enumerate(df['Lyric']):
    tokens = tokenizer.encode_plus(lyric, max_length=seq_len, truncation=True, #truncation_True -> text longer than 512 tokens get compressed
                                   padding='max_length', add_special_tokens=True, #for text shorter than 512, pad to length 512. Special tokens adds [CLS],[SEP],[PAD]
                                   return_tensors='tf')
    # assign tokenized outputs to respective rows in numpy arrays
    Xids[i, :] = tokens['input_ids']
    Xmask[i, :] = tokens['attention_mask']

(3000, 512)


101 is for [CLS] "start of seq", 0 is for [PAD] , 

In [4]:
Xids

array([[  101.,   164.,  1130., ...,  2552.,   117.,   102.],
       [  101.,  1130.,   170., ...,  4932.,  1122.,   102.],
       [  101.,  1192.,  1363., ...,   146., 16804.,   102.],
       ...,
       [  101.,  1409.,   146., ...,     0.,     0.,     0.],
       [  101.,   138., 13782., ...,     0.,     0.,     0.],
       [  101.,  1192.,  1686., ...,     0.,     0.,     0.]])

In [5]:
Xmask

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

Xmask is a control for the attention layer in bert. whereever xmask is 1 Bert will calculate the attention for that token. To avoid for Bert making connections with padding tokens



### one-hot encoding the Genres

In [6]:

# first extract sentiment column
arr = df['label'].values
# we then initialize the zero array
labels = np.zeros((num_samples, arr.max()+1))

# set relevant index for each row to 1 (one-hot encode)
labels[np.arange(num_samples), arr] = 1
print(labels)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


### Building a tf.data.Dataset object using the input and label tensors. Then transforming them into the correct format for the model.

In [7]:
# create the dataset object
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# then we use the dataset map method to apply this transformation
dataset = dataset.map(map_func)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [8]:
# we will split into batches of 16
batch_size = 16

# shuffle and batch - dropping any remaining samples that don't cleanly
# fit into a batch of 16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
dataset.take(1) # We now have 16 samples per batch

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 3)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [9]:
# set split size (90% training data) and calculate training set size
split = 0.9
size = int((Xids.shape[0]/batch_size)*split)
print(size)
print('number of samples in training ' + str(56*16))
# get training and validation sets
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

168
number of samples in training 896


In [10]:
val_ds

<SkipDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 3)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [11]:
val_ds.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 3)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [12]:
# AutoModel for PyTorch, TFAutoModel for TensorFlow
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [13]:
import tensorflow as tf

# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert.bert(input_ids, attention_mask=mask)[0]  # access final activations with [0]

# convert bert embeddings into 3 output classes
x = tf.keras.layers.LSTM(32, dropout=.3, recurrent_dropout=.3, return_sequences=True)(embeddings)
x = tf.keras.layers.LSTM(16, dropout=.4, recurrent_dropout=.4, return_sequences=False)(x)
# normalize
x = tf.keras.layers.BatchNormalization()(x)
# output
x = tf.keras.layers.Dense(64, activation='relu')(x)
y = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(x)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [14]:
# initialize model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# (optional) freeze bert layer
model.layers[2].trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [15]:
optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [16]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2
)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
168/168 [==============================] - ETA: 0s - loss: 1.1394 - accuracy: 0.3248WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fe63d9aacb0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and

In [17]:
pred = model.predict(val_ds)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [18]:
y = np.concatenate([labels for Lyric, labels in val_ds], axis=0)
y = np.argmax(y, axis=1)

In [19]:
pred
y_pred_bool = np.argmax(pred, axis=1)
y_pred_bool

array([1, 1, 2, 2, 2, 2, 0, 2, 0, 0, 1, 0, 2, 2, 2, 1, 0, 2, 0, 0, 2, 2,
       2, 1, 2, 1, 2, 2, 2, 1, 0, 2, 0, 0, 1, 1, 2, 2, 1, 1, 0, 2, 1, 2,
       0, 1, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 1, 2, 2, 1, 2,
       2, 1, 0, 1, 1, 0, 2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2,
       2, 0, 2, 0, 2, 1, 0, 2, 1, 2, 0, 1, 2, 2, 1, 1, 0, 1, 1, 0, 2, 1,
       0, 1, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 1, 2, 0, 1, 2, 1,
       0, 1, 0, 0, 0, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 0, 1, 1, 1, 1, 2,
       0, 2, 2, 1, 2, 2, 1, 0, 2, 1, 2, 2, 1, 2, 1, 0, 2, 2, 2, 1, 0, 1,
       1, 0, 0, 2, 1, 1, 0, 2, 2, 0, 2, 2, 0, 0, 1, 2, 2, 0, 0, 2, 2, 2,
       2, 2, 2, 1, 0, 0, 0, 2, 1, 1, 2, 2, 1, 2, 1, 2, 0, 0, 2, 2, 0, 1,
       1, 2, 0, 0, 0, 2, 2, 1, 2, 2, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 0,
       1, 2, 1, 2, 1, 2, 2, 2, 2, 0, 2, 2, 1, 0, 2, 1, 1, 1, 2, 1, 2, 2,
       2, 2, 0, 1, 2, 1, 1, 2, 1, 0, 1, 1, 1, 2, 1,

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y, y_pred_bool, target_names=label_dict))

              precision    recall  f1-score   support

     Hip Hop       0.37      0.31      0.33        95
         Pop       0.42      0.37      0.39       106
        Rock       0.35      0.45      0.39       103

    accuracy                           0.38       304
   macro avg       0.38      0.37      0.37       304
weighted avg       0.38      0.38      0.37       304

